In [ ]:
import tensorflow as tf 
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.optimizers as optim

import pandas as pd 
import numpy as np 
import os

In [ ]:
PATH = r'.../tox21_dense_train.csv'
data = pd.read_csv(PATH).values
labels = pd.read_csv(PATH).columns
labels

In [ ]:
for x in range(799):
    if max(data[:, 3:][:, x])!=min(data[:, 3:][:, x]):
        xtr = (data[:, 3:][:, x]-np.mean(data[:, 3:][:, x]))/(max(data[:, 3:][:, x])-min(data[:, 3:][:, x]))
        data[:, 3:][:, x] = xtr
    else:
        xtr = (data[:, 3:][:, x])/(max(data[:, 3:][:, x])+min(data[:, 3:][:, x])+1e-10)+0.5
        data[:, 3:][:, x] = xtr

In [ ]:
data = data[:, 3:]

In [ ]:
xtrain = np.array(data[:10000], dtype="float32")
xvalid = np.array(data[10000:12000], dtype="float32")

In [ ]:
def DenseBlock(x, ndim):
    x = layers.Dense(ndim)(x)
    x = layers.BatchNormalization(momentum=0.5)(x)
    x = layers.LeakyReLU(0.2)(x)
    return  x

def autoencoder(inp):
    inp = layers.Input(inp)
    x = DenseBlock(inp, 256)
    enc_out = DenseBlock(x, 100)
    x = DenseBlock(enc_out, 256)
    dec_out = DenseBlock(x, 799)
    encoder = models.Model(inputs=inp, outputs=enc_out, name="Encoder")
    model = models.Model(inputs=inp, outputs=dec_out, name="Autoencoder")

    return encoder, model

encoder, model = autoencoder((799))
print(encoder.summary())
print(model.summary())

In [ ]:
model.compile(loss='mse', optimizer="adam", metrics="accuracy")
model.fit(xtrain, xtrain, batch_size=100, verbose=1, epochs=100, validation_data=[xvalid, xvalid], validation_batch_size=100)

In [ ]:
model.save("./model.h5")
encoder.save("./encoder.h5")